In [120]:
import os
from Levenshtein import ratio, jaro_winkler
import pandas as pd
import numpy as np

In [49]:
gold_standard = os.path.join('thesis_data', 'vangogh_gold_standard')

In [57]:
def match_function(x, y):
    if ratio(x, y) > 0.7:
        return True
    elif ratio(x, y) > 0.5 and jaro_winkler(x, y) > 0.5:
        return True
    
    else:    
        return False

In [40]:
def merge_function(x, y):
    if len(x) > len(y):
        return x
    else:
        return y

In [144]:
which = 'let027'
gs_data = pd.read_csv(os.path.join(gold_standard, '{}.csv'.format(which)))
gs_data.sort_values(by=['start_offsets'], inplace=True)
which2 = 'let028'
gs_data2 = pd.read_csv(os.path.join(gold_standard, '{}.csv'.format(which2)))
gs_data = gs_data.append(gs_data2, ignore_index=True)
gs_data

,start_offsets,end_offsets,keys,types,entities
0,62,70,707,pers,Michelet
1,740,744,412,pers,Anna
2,940,942,526,pers,Pa
3,983,985,524,pers,Ma
4,1273,1284,398,pers,Grandfather
5,1336,1341,499 501,pers,aunts
6,1485,1489,412,pers,Anna
7,1723,1737,442 443,pers,the Haanebeeks
8,1831,1842,1202,pers,Thijs Maris
9,1848,1859,787,pers,Mr Tersteeg


In [145]:
entities = list(gs_data['entities'].values)
print(entities)

['Michelet', 'Anna', 'Pa', 'Ma', 'Grandfather', 'aunts', 'Anna', 'the Haanebeeks', 'Thijs Maris', 'Mr Tersteeg', 'Mr Obach', 'Anna', 'Boughton', 'Maris', 'Jacquet', 'Anna', 'Mr T', 'Michelet', 'Ary Scheffer', 'Leys', 'Tissot', 'Fagel', 'Leys', 'Alphonse Karr', 'Anna']


In [146]:
I = entities.copy()
I_prime = []
while len(I) > 0:
    current_record = I.pop(0)
    buddy = ''
    for r_ix in range(0, len(I_prime)):
        if match_function(current_record, I_prime[r_ix]):
            buddy = I_prime[r_ix]
            break
        else:
            pass
    if len(buddy) == 0:
        I_prime.append(current_record)
    else:
        new_record = merge_function(current_record, buddy)
        I_prime.pop(r_ix)
        I.append(new_record)
        
for e in entities:
    sim_vec = []
    for ix in range(0, len(I_prime)):
        sim_vec.append(ratio(e, I_prime[ix]))
    max_ix = np.argmax(sim_vec)
    print(e, '->', I_prime[max_ix])    
        

Michelet -> Michelet
Anna -> Anna
Pa -> Pa
Ma -> Pa
Grandfather -> Grandfather
aunts -> aunts
Anna -> Anna
the Haanebeeks -> the Haanebeeks
Thijs Maris -> Thijs Maris
Mr Tersteeg -> Mr Tersteeg
Mr Obach -> Mr Obach
Anna -> Anna
Boughton -> Boughton
Maris -> Thijs Maris
Jacquet -> Jacquet
Anna -> Anna
Mr T -> Mr Tersteeg
Michelet -> Michelet
Ary Scheffer -> Ary Scheffer
Leys -> Leys
Tissot -> Tissot
Fagel -> Fagel
Leys -> Leys
Alphonse Karr -> Alphonse Karr
Anna -> Anna


In [111]:
merge_function('Maris', 'Ma')

'Maris'